# Algorithmic Fairness, Accountability, and Ethics, Spring 2024

## Mandatory Assignment 2

Please use the following code to prepare the dataset.
 

In [ ]:
from folktables.acs import adult_filter
from folktables import ACSDataSource, BasicProblem, generate_categories
import numpy as np
from scipy.stats import pearsonr
import pandas as pd
import random
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings("ignore")
random.seed(12)

data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["CA"], download=True)


def adult_filter(data):
    """Mimic the filters in place for Adult data.
    Adult documentation notes: Extraction was done by Barry Becker from
    the 1994 Census database. A set of reasonably clean records was extracted
    using the following conditions:
    ((AAGE>16) && (AGI>100) && (AFNLWGT>1)&& (HRSWK>0))
    """
    df = data
    df = df[df['AGEP'] > 16]
    df = df[df['PINCP'] > 100]
    df = df[df['WKHP'] > 0]
    df = df[df['PWGTP'] >= 1]
    df = df[df["RAC1P"] < 3] ## keep only Whites and African-Americans
    return df


ACSIncomeNew = BasicProblem(
    features=[
        'AGEP',
        'COW',
        'SCHL',
        'MAR',
        'RELP',
        'WKHP',
        'PWGTP',
        'SEX',
        'RAC1P',
    ],
    target='PINCP',
    target_transform=lambda x: x > 25000,    
    group=['SEX', 'RAC1P'],
    preprocess=adult_filter,
    postprocess=lambda x: np.nan_to_num(x, -1),
)

definition_df = data_source.get_definitions(download=True)
categories = generate_categories(features=ACSIncomeNew.features, definition_df=definition_df)
features, labels, groups = ACSIncomeNew.df_to_pandas(acs_data, categories=categories, dummies=True)

# Drop the "redundant" columns
features = features.drop(["RAC1P_White alone", 
                          "SEX_Male", 
                          "SCHL_1 or more years of college credit, no degree",  
                          "MAR_Divorced", 
                          "RELP_Adopted son or daughter",
                          'COW_Working without pay in family business or farm' ], axis = 1) 

print("Columns with the protected features:")
for i, f in enumerate(features.columns):
    if ("RAC1P" in f) or ("SEX" in f):
        print("Column ID: %s" %i, "(%s)"%f)
        
features.head()

In [ ]:
### Train test split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

sample_indices = random.sample(range(len(features)), 20000)
features, labels = features.iloc[sample_indices], labels.iloc[sample_indices]

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)
scaler = StandardScaler()
scaler.fit(X_train)
X_train = pd.DataFrame(columns= X_train.columns, data= scaler.transform(X_train), index = X_train.index)
X_test = pd.DataFrame(columns= X_test.columns, data = scaler.transform(X_test), index= X_test.index)

### Create one classifier to predict income on RAW DATA
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1)
gbc.fit(X_train, y_train)

### Report general accuracy 
from sklearn.metrics import accuracy_score, confusion_matrix, auc, roc_curve
from sklearn.model_selection import cross_val_score

probs = gbc.predict_proba(X_test)[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, probs)
preds = accuracy_score(y_test, probs > 0.5)

print('Scores across all groups.')
print('AUC: ', auc(fpr, tpr))
print('Accuracy: ', preds)
print('Cross val score: ', cross_val_score(gbc, X_test, y_test))
print('---------------------------')

### report accuracy for gender groups and races

for i in set(X_test['SEX_Female']):
    X_test_temp = X_test[X_test["SEX_Female"] == i]
    drops = []
    for j in y_test.index:
        if j not in X_test_temp.index:
            drops.append(j)
    y_test_temp = y_test.drop(drops)
    temp_preds = gbc.predict(X_test_temp)
    accuracy = accuracy_score(y_test_temp, temp_preds)
    print(f'Accuracy for SEX {i}: {accuracy}')
    print('Cross val score: ', cross_val_score(gbc, X_test, y_test))
    print('---------------------------')

race_columns = [col for col in X_test.columns if "RAC1P" in col]

for i in set(X_test['RAC1P_Black or African American alone']):
    X_test_temp = X_test[X_test['RAC1P_Black or African American alone'] == i]
    drops = []
    for j in y_test.index:
        if j not in X_test_temp.index:
            drops.append(j)
    y_test_temp = y_test.drop(drops)
    temp_preds = gbc.predict(X_test_temp)
    accuracy = accuracy_score(y_test_temp, temp_preds)
    print(f'Accuracy for RAC1P {i}: {accuracy}')
    print('Cross val score: ', cross_val_score(gbc, X_test_temp, y_test_temp))
    print('---------------------------')

In [ ]:
Xs_train_p = X_train.values[:, 54:]
Xs_test_p = X_test.values[:, 54:]
Xs_train_np = X_train.values[:, :54]
Xs_test_np = X_test.values[:, :54]

In [ ]:
# ### Create a fairer version of the dataset to protect select groups
# protected_cols = ['RAC1P_Black or African American alone','SEX_Female']
# X_train_unprotected, X_train_protected = X_train.drop(columns = protected_cols), X_train[protected_cols]
# X_test_unprotected, X_test_protected = X_test.drop(columns = protected_cols), X_test[protected_cols]

def debias_features(Xs_np, Xs_p, lambda_param=0.7):
    import scipy
    assert Xs_np.shape[0]==Xs_p.shape[0]
    
    # Find orthonormal basis of protected features
    orthbasis = scipy.linalg.orth(Xs_p)

    # Debias nonprotected features
    Xs_np_debiased = Xs_np - lambda_param * (orthbasis @ orthbasis.T @ Xs_np)

    # Return debiased nonprotected features
    return Xs_np_debiased
accuracies_all = []
accuracies_sex_1 = []
accuracies_sex_2 = []
accuracies_race_1 = []
accuracies_race_2 = []
protected_cols = ['RAC1P_Black or African American alone', 'SEX_Female']

for idx, i in enumerate([0.1, 1/3, 0.5, 2/3, 1]):
    X_train_unprotected_debiased = debias_features(Xs_train_np, Xs_train_p, i)
    X_train_debiased = np.concatenate([X_train_unprotected_debiased, Xs_train_p], axis=1)
    # Compute correlation matrix
    n_features = X_train.shape[1]
    corr_ = np.zeros((n_features, n_features))
    p_ = np.zeros((n_features, n_features))
    for i in range(n_features):
        for j in range(n_features):
            corr_[i,j], p_[i,j] = pearsonr(X_train_debiased[:,i], X_train_debiased[:,j])
            corr_ = np.nan_to_num(corr_, 0)

    # Plot correlations with protected features
    plt.figure(idx, figsize=(4,15))
    sns.heatmap(corr_[:,54:], cmap="bwr", xticklabels=features.columns[54:], yticklabels=features.columns, vmin=-1, vmax=1)
    plt.title("Pearson's Correlation Coeff between SEX and other variables (Masked by p value)")
    plt.show()
    # Train a classifier on the debiased data for each group
    gbc.fit(X_train_debiased, y_train)
    probs = gbc.predict_proba(X_test)[:, 1]
    fpr, tpr, thresholds = roc_curve(y_test, probs)
    accuracy = accuracy_score(y_test, probs > 0.5)
    accuracies_all.append(accuracy)
    X_test_women = X_test[X_test['SEX_Female'] > 0]
    X_test_men = X_test[X_test['SEX_Female'] < 0]
    y_test_men = y_test.loc[X_test_men.index]
    y_test_women = y_test.loc[X_test_women.index]
    X_test_black = X_test[X_test['RAC1P_Black or African American alone'] > 0]
    X_test_non_black = X_test[X_test['RAC1P_Black or African American alone'] < 0]
    y_test_black = y_test.loc[X_test_black.index]
    y_test_non_black = y_test.loc[X_test_non_black.index]
    accuracies_race_1.append(accuracy_score(y_test_black, gbc.predict(X_test_black)))
    accuracies_race_2.append(accuracy_score(y_test_non_black, gbc.predict(X_test_non_black)))
    accuracies_sex_1.append(accuracy_score(y_test_women, gbc.predict(X_test_women)))
    accuracies_sex_2.append(accuracy_score(y_test_men, gbc.predict(X_test_men)))

In [ ]:
stats = pd.DataFrame()
stats.index = [0.1, 1/3, 0.5, 2/3, 1]
stats['All'] = accuracies_all
stats['Women'] = accuracies_sex_1
stats['Men'] = accuracies_sex_2
stats['Black'] = accuracies_race_1
stats['Non-Black'] = accuracies_race_2

In [ ]:
plt.figure(figsize=(10, 5))
sns.lineplot(data=stats, x = stats.index, y = stats['All'], label = 'All', markers=True)
sns.lineplot(data=stats, x = stats.index, y = 'Men', label = 'men', markers=True)
sns.lineplot(data=stats, x = stats.index, y = 'Women', label = 'Women', markers=True)
sns.lineplot(data=stats, x = stats.index, y = 'Black', label = 'Black', markers=True)
sns.lineplot(data=stats, x = stats.index, y = 'Non-Black', label = 'Non-Black', markers=True)

In [ ]:
### Build model using de-correlation effect from https://dl.acm.org/doi/10.1145/3375627.3375864

### Record results and plot differences in accuracies


# Task 2.2 - FairPCA

## Reprojecting data using FairPCA implementation from exercise 7 solution

In [ ]:
import scipy

In [ ]:
class FairPCA:
    def __init__(self, Xs, p_idxs, n_components):
        self.fit(Xs, p_idxs, n_components)

    def fit(self, Xs, p_idxs, n_components):
        # Extract protected features
        Xs_p = Xs[:, p_idxs]

        # Compute projection matrix (U)
        Z = Xs_p
        #Z = Z - Z.mean(0) # Since we alredy standardised everything, there is not much sense in removing the mean
        R = scipy.linalg.null_space(Z.T @ Xs)
        eig_vals, L = scipy.linalg.eig(R.T @ Xs.T @ Xs @ R)
        self.U = R @ L[:, :n_components]

    def project(self, Xs):
        return Xs @ self.U
    
fair_pca = FairPCA(X_train.to_numpy(), [54, 55], 30)
Xs_train_debiased = fair_pca.project(X_train)
Xs_test_debiased = fair_pca.project(X_test)

In [ ]:
# Compute correlation matrix
Xs_train_debiased_p = np.concatenate([Xs_train_debiased, Xs_train_p], axis=1)
n_features = Xs_train_debiased_p.shape[1]
corr_ = np.zeros((n_features, n_features))
p_ = np.zeros((n_features, n_features))
for i in range(n_features):
    for j in range(n_features):
        corr_[i,j], p_[i,j] = pearsonr(Xs_train_debiased_p[:,i], Xs_train_debiased_p[:,j])
        corr_ = np.nan_to_num(corr_, 0)

# Plot correlations with protected features
# plt.figure(figsize=(4,15))
# sns.heatmap(corr_[:,-2:], cmap="coolwarm", xticklabels=features.columns[-2:], vmin=-1, vmax=1)
# plt.title("Pearson's Correlation Coeff between SEX and components (Masked by p value)")
# plt.show()

## Building classification model with reprojected data from FairPCA

In [ ]:
# Adapted from previous code, some fuckery had to happen

# convert to pandas DataFrames
Xs_train_debiased_p_df = pd.DataFrame(data=Xs_train_debiased_p)
Xs_test_debiased_p = np.concatenate([Xs_test_debiased, Xs_test_p], axis=1)
Xs_test_debiased_p_df = pd.DataFrame(data=Xs_test_debiased_p)

# overall accuracy
gbc.fit(Xs_train_debiased_p_df.drop([30, 31], axis=1), y_train)
probs = gbc.predict_proba(Xs_test_debiased_p_df.drop([30, 31], axis=1))[:, 1]
fpr, tpr, thresholds = roc_curve(y_test, probs)
accuracy = accuracy_score(y_test, probs > 0.5)
print(f'Overall accuracy:   {accuracy}')

# accuracy for the groups
Xs_test_debiased_women = Xs_test_debiased_p_df[Xs_test_debiased_p_df[30] > 0]
Xs_test_debiased_men = Xs_test_debiased_p_df[Xs_test_debiased_p_df[30] < 0]
y_test_men = y_test.iloc[Xs_test_debiased_men.index]
y_test_women = y_test.iloc[Xs_test_debiased_women.index]
Xs_test_debiased_black = Xs_test_debiased_p_df[Xs_test_debiased_p_df[31] > 0]
Xs_test_debiased_non_black = Xs_test_debiased_p_df[Xs_test_debiased_p_df[31] < 0]
y_test_black = y_test.iloc[Xs_test_debiased_black.index]
y_test_non_black = y_test.iloc[Xs_test_debiased_non_black.index]
print(f'Accuracy Black:     {accuracy_score(y_test_black, gbc.predict(Xs_test_debiased_black.drop([30, 31], axis=1)))}')
print(f'Accuracy Non-Black: {accuracy_score(y_test_non_black, gbc.predict(Xs_test_debiased_non_black.drop([30, 31], axis=1)))}')
print(f'Accuracy Women:     {accuracy_score(y_test_women, gbc.predict(Xs_test_debiased_women.drop([30, 31], axis=1)))}')
print(f'Accuracy Men:       {accuracy_score(y_test_men, gbc.predict(Xs_test_debiased_men.drop([30, 31], axis=1)))}')